In [27]:
import pandas as pd

In [28]:
import pandas as pd
from scipy import stats
import numpy as np
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
X_train = pd.read_csv('X_train_prep')

In [4]:
X_test = pd.read_csv('X_test_prep')

In [35]:
X= pd.read_csv('/Users/mi0310/Desktop/JPMC/train.csv')
Y_train = X["Fare"]

In [9]:
X["To"].value_counts()

Mumbai       1447
Delhi        1424
Lucknow      1420
Patna         872
Chennai       809
Kolkata       770
Hyderabad     758
Name: To, dtype: int64

In [47]:
distance = pd.read_csv('/Users/mi0310/Downloads/dist (1).csv')

In [48]:
city_map = {"Delhi": 1, "Mumbai": 2, "Lucknow": 3, "Kolkata": 4, "Hyderabad": 5, "Patna":6, "Chennai":7}

In [49]:
distance["From"] = distance["from"]
distance["To"] = distance["to"]

In [50]:
del distance["from"]
del distance["to"]

In [51]:
distance

,distance,From,To
0,1446.50,Lucknow,Mumbai
1,1419.33,Delhi,Mumbai
2,710.00,Hyderabad,Mumbai
3,2054.77,Kolkata,Mumbai
4,1744.17,Patna,Mumbai
5,1334.00,Chennai,Mumbai
6,536.00,Delhi,Lucknow
7,1434.00,Hyderabad,Lucknow
8,991.00,Kolkata,Lucknow
9,539.00,Patna,Lucknow


In [52]:
distance['From'] = distance['From'].map(city_map)
distance['To'] = distance['To'].map(city_map)

In [53]:
distance

,distance,From,To
0,1446.50,3,2
1,1419.33,1,2
2,710.00,5,2
3,2054.77,4,2
4,1744.17,6,2
5,1334.00,7,2
6,536.00,1,3
7,1434.00,5,3
8,991.00,4,3
9,539.00,6,3


In [54]:
s1 = pd.merge(X_train, distance, how='left', on=['From', 'To'])

In [56]:
s2 = pd.merge(X_test, distance, how='left', on=['From', 'To'])

In [129]:
s1.dtypes

From                        int64
To                          int64
Class                       int64
Age_at_Booking              int64
Booking_to_flight           int64
Booking Year                int64
Booking Month               int64
Booking Dayofweek           int64
Booking Is_month_end         bool
Booking Is_month_start       bool
Flight Month                int64
Flight Dayofweek            int64
Flight Is_month_end          bool
Flight Is_month_start        bool
Time Label                  int64
distance                  float64
dtype: object

In [70]:
X_train = s1
X_test = s2

In [73]:
del X_train["Flight Time"]
del X_train["Flight Year"]

In [147]:
cols_to_transform = ['Class', 'From', 'To', 'Time Label']
X = pd.get_dummies(X, columns=cols_to_transform)

In [78]:
cols_to_transform = ['Class', 'From', 'To', 'Time Label']
X_test = pd.get_dummies(X_test, columns=cols_to_transform)

In [84]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=25, random_state=0,n_estimators=500)

In [94]:
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [95]:
Y_pred = regr.predict(X_test)

In [96]:
Y_pred

array([ 1736.15246,  2116.23454,  7665.24192, ...,  7981.8579 ,
        7408.52574, 11156.16658])

In [98]:
np.savetxt("y_pred_randomforest_distancesquared.csv", Y_pred, delimiter=",")

In [89]:
X_train.to_csv('X_preprocessed.csv')

In [90]:
X_test.to_csv('X_test_preprocessed.csv')

In [92]:
X_train["squaredistance"] = pow(X_train["distance"],2)

In [93]:
X_test["squaredistance"] = pow(X_test["distance"],2)

In [109]:
X_wd = X_train["distance"]

In [107]:
X_wd = pd.DataFrame()

In [111]:
X_wd["distance"] = X_train["distance"]

In [115]:
regr.fit(X, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [117]:
Y_pred = regr.predict(X_test[["distance","squaredistance"]])

In [118]:
np.savetxt("y_pred_randomforest_distancesquared_less.csv", Y_pred, delimiter=",")

In [120]:
X_train["Fare"] = Y_train

In [130]:
s1["Fare"] = Y_train

In [145]:
X = s1[s1["From"]==8]
for i in range(1,8):
    for j in range(1,8):
        X2 = s1[s1["From"]==i]
        X2 = X2[X2["To"]==j]
        if(len(X2)>0):
            low = np.percentile(X2["Fare"],1.5)
            high = np.percentile(X2["Fare"],98.5)
            X2 = X2[X2["Fare"]>=low]
            X2 = X2[X2["Fare"]<=high]
            print(len(X2))
        X=X.append(X2)

337
334
168
182
196
183
348
344
174
160
181
166
357
335
147
154
173
181
158
207
161
84
103
81
192
158
171
89
93
87
163
185
191
68
76
81
158
175
171
94
72
94


In [148]:
X.dtypes

Age_at_Booking              int64
Booking_to_flight           int64
Booking Year                int64
Booking Month               int64
Booking Dayofweek           int64
Booking Is_month_end         bool
Booking Is_month_start       bool
Flight Month                int64
Flight Dayofweek            int64
Flight Is_month_end          bool
Flight Is_month_start        bool
distance                  float64
Fare                      float64
Class_0                     uint8
Class_1                     uint8
From_1                      uint8
From_2                      uint8
From_3                      uint8
From_4                      uint8
From_5                      uint8
From_6                      uint8
From_7                      uint8
To_1                        uint8
To_2                        uint8
To_3                        uint8
To_4                        uint8
To_5                        uint8
To_6                        uint8
To_7                        uint8
Time Label_0  

In [149]:
Y = X["Fare"]
del X["Fare"]

In [142]:
del X_test["squaredistance"]

In [143]:
del X["Fare"]

In [150]:
regr.fit(X, Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [151]:
Y_pred = regr.predict(X_test)
np.savetxt("y_pred_randomforest_distancesquared_less_outliersno.csv", Y_pred, delimiter=",")

In [155]:
len(Y_train)

7500

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [75, 100],'max_features':[10, 20],'max_depth':[20, 30]}]
forest_reg = RandomForestRegressor(n_jobs=-1)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train,Y_train)

In [159]:
regr = RandomForestRegressor(max_depth=20, max_features=20,  min_samples_leaf=1,random_state=0,n_estimators=100)

In [158]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [166]:
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [167]:
Y_pred = regr.predict(X_test)
np.savetxt("y_pred_randomforest_distancesquared_less_outliersno_gridsearch.csv", Y_pred, delimiter=",")

In [163]:
X_test.dtypes

Age_at_Booking              int64
Booking_to_flight           int64
Booking Year                int64
Booking Month               int64
Booking Dayofweek           int64
Booking Is_month_end         bool
Booking Is_month_start       bool
Flight Month                int64
Flight Dayofweek            int64
Flight Is_month_end          bool
Flight Is_month_start        bool
distance                  float64
Class_0                     uint8
Class_1                     uint8
From_1                      uint8
From_2                      uint8
From_3                      uint8
From_4                      uint8
From_5                      uint8
From_6                      uint8
From_7                      uint8
To_1                        uint8
To_2                        uint8
To_3                        uint8
To_4                        uint8
To_5                        uint8
To_6                        uint8
To_7                        uint8
Time Label_0                uint8
Time Label_1  

In [165]:
del X_train["Fare"]
del X_train["squaredistance"]

In [30]:
X_test = pd.read_csv('X_preprcessed_test.csv')

In [31]:
X_train = pd.read_csv('X_train_preprcessed.csv')

In [32]:
del X_train["Unnamed: 0"]
del X_test["Unnamed: 0"]

In [171]:
plot_importance(regr)
pyplot.show()

NameError: name 'plot_importance' is not defined

In [8]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier


xgbc = XGBClassifier(**xgb_params, verbose_eval=4)

In [ ]:
xgbc.fit(X_train,Y_train)

In [191]:
X_tr.dtypes

Age_at_Booking            int64
Booking_to_flight         int64
Booking Year              int64
Booking Month             int64
Booking Dayofweek         int64
Booking Is_month_end      int64
Booking Is_month_start    int64
Flight Month              int64
Flight Dayofweek          int64
Flight Is_month_end       int64
Flight Is_month_start     int64
distance                  int64
Class_0                   int64
Class_1                   int64
From_1                    int64
From_2                    int64
From_3                    int64
From_4                    int64
From_5                    int64
From_6                    int64
From_7                    int64
To_1                      int64
To_2                      int64
To_3                      int64
To_4                      int64
To_5                      int64
To_6                      int64
To_7                      int64
Time Label_0              int64
Time Label_1              int64
Time Label_2              int64
Time Lab

In [192]:
len(Y_train)

7500

In [36]:
xgb_params = {
    'max_depth': 9, 
    'n_estimators' : 500,
    'learning_rate': 0.04,
    'min_child_weight' : 2,
    'objective':'reg:linear',
     'colsample_bylevel':0.8,
     'colsample_bytree':0.8,
    'silent': False,
}
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, Y_train)
dtest=xgb.DMatrix(X_test)
model=xgb.train(xgb_params,dtrain, num_boost_round=178)
preds = model.predict(dtest)

In [37]:
np.savetxt("Y_preddd_xgboosterrr.csv", preds, delimiter=",")

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4],'n_estimators' : [100,200,300,400]}

xgbc = xgb.XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgbc, params)
grid.fit(X_train, Y_train)

In [22]:
import pandas as pd
pred1 = pd.read_csv('Y_preddd_xgboost.csv')
pred2 = pd.read_csv('y_pred_randomforest_distancesquared_less_outliersno_gridsearch.csv')

In [26]:
import numpy as np
np.savetxt("xgboost+randomforest.csv", pred, delimiter=",")

In [14]:
pred1 = pred1.astype(float)

In [24]:
pred = pred1+pred2

In [25]:
pred

,0
0,3651.401401
1,4154.380712
2,14870.034862
3,9221.935714
4,16812.005388
5,17639.838506
6,20820.916803
7,10086.338931
8,10536.819172
9,6279.574559


In [39]:
from numpy import loadtxt
from xgboost import XGBClassifier
from matplotlib import pyplot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

AttributeError: 'Booster' object has no attribute 'feature_importances_'

In [40]:
print(model.feature_importances_)

AttributeError: 'Booster' object has no attribute 'feature_importances_'

In [41]:
X_train.dtypes

Age_at_Booking              int64
Booking_to_flight           int64
Booking Year                int64
Booking Month               int64
Booking Dayofweek           int64
Booking Is_month_end         bool
Booking Is_month_start       bool
Flight Month                int64
Flight Dayofweek            int64
Flight Is_month_end          bool
Flight Is_month_start        bool
distance                  float64
Class_0                     int64
Class_1                     int64
From_1                      int64
From_2                      int64
From_3                      int64
From_4                      int64
From_5                      int64
From_6                      int64
From_7                      int64
To_1                        int64
To_2                        int64
To_3                        int64
To_4                        int64
To_5                        int64
To_6                        int64
To_7                        int64
Time Label_0                int64
Time Label_1  